# 深度学习计算2_模型参数访问、初始化和共享           

定义一个含单隐藏层的多层感知机。使用**默认方式**初始化它的参数，并做一次前向计算。与之前不同的是，在这里从`nn`中导入了`init模块`，它包含了多种模型初始化方法。

In [1]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)

Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 访问模型参数
由于`Sequential类`与`Module类`的**继承关系**。对于`Sequential实例`中含模型参数的层，可以通过`Module类`的`parameters()`或者`named_parameters`方法来访问所有参数（以迭代器的形式返回），后者除了返回参数Tensor外还会返回其名字。

In [2]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


返回的名字自动加上了层数的索引作为前缀。         

访问`net`中**单层**的参数。对于使用`Sequential类`构造的神经网络，我们可以通过方括号[]来访问网络的任一层。索引`0`表示隐藏层为`Sequential`实例最先添加的层。

In [3]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


因为这里是单层的所以没有了层数索引的前缀。另外返回的`param`的类型为`torch.nn.parameter.Parameter`，其实这是`Tensor的子类`，和Tensor不同的是如果一个`Tensor`是`Parameter`，那么它**会自动被添加到模型的参数列表里**

In [4]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20)) # 被自动添加进入模型参数
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


上面的代码中`weight1`在参数列表中但是`weight2`却没在参数列表中。      

因为`Parameter`是`Tensor`，即`Tensor`拥有的属性它都有，比如可以根据`data`来访问参数数值，用`grad`来访问参数梯度。

In [5]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[-0.4788, -0.4901, -0.2424, -0.1711],
        [ 0.3348,  0.0330, -0.2703, -0.4362],
        [-0.2837,  0.2228,  0.3073,  0.4925]])
None
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.1073, 0.5128, 0.0696, 0.4335]])


## 初始化模型参数   

PyTorch中nn.Module的模块参数都采取了较为合理的初始化策略（不同类型的layer具体采样的哪一种初始化方法的可参考源代码）

In [6]:
## 将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

# 下面使用常数来初始化权重参数。

for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.weight tensor([[-0.0090, -0.0185, -0.0036,  0.0056],
        [-0.0069, -0.0015,  0.0055, -0.0003],
        [-0.0012,  0.0111,  0.0004,  0.0006]])
2.weight tensor([[ 0.0169, -0.0152, -0.0052]])
0.bias tensor([0., 0., 0.])
2.bias tensor([0.])



## 自定义初始化方法

可以看到这就是一个`inplace`改变`Tensor`值的函数，而且**这个过程是不记录梯度的**。

类似的我们来实现一个自定义的初始化方法。    

在下面的例子里，我们令权重有一半概率初始化为`0`，有另一半概率初始化为`[−10,−5]`和`[5,10]`两个区间里均匀分布的随机数。

In [7]:
# 不记录梯度
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)
    

# 自定义初始化
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-6.5661, -7.7167,  0.0000,  5.0827],
        [ 9.8248,  7.1661,  0.0000, -8.5346],
        [-8.8847, -6.1681,  0.0000, -7.4899]])
2.weight tensor([[-0.0000, 6.1088, 5.0403]])


还可以通过改变这些参数的`data`来改写模型参数值同时**不会影响梯度**:

In [8]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 共享模型参数    

在有些情况下，希望在多个层之间**共享模型参数**。其中一种方法: `Module类`的`forward函数`里多次**调用同一个层**。       

此外，如果我们传入`Sequential`的模块是同一个`Module`实例的话参数也是共享的:


In [9]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    # w初始化为3
    init.constant_(param, val=3)
    print(name, param.data)
    


Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


在内存中，这两个线性层其实是一个对象

In [10]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


因为模型参数里包含了梯度，所以在反向传播计算时，这些**共享的参数的梯度是累加的**

In [11]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次梯度是3，两次所以就是6


tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
